In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from scipy.stats import mode

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [12]:
# Import .csv files
df_portfolio = pd.read_csv('/content/portfolio.csv', index_col=0)
df_profile = pd.read_csv('/content/profile.csv', index_col=0)
df_transcript = pd.read_csv('/content/transcript.csv', index_col=0)

In [13]:
# Preview portfolio.csv
print(df_portfolio.shape)
df_portfolio.head()

(10, 6)


,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [14]:
# Preview profile.csv
print(df_profile.shape)
df_profile.head()

# Check df1['id'] for duplicates
df_profile.duplicated('id').any()



(17000, 5)


False

In [15]:
# Preview transcript.csv
print(df_transcript.shape)
df_transcript.head()

(306534, 4)


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


There are no duplicated customer ID numbers in <code>df1</code>. This means that each person in the dataframe is unique.

In [16]:
# Verify matching set of customer IDs
if len(set(df_profile['id']) ^ set(df_transcript['person'])) == 0:
    print("The dataframes df1 and df2 are comprised of the same individuals.")
else:
    print("The people in df1 and those in df2 do not match.")

The dataframes df1 and df2 are comprised of the same individuals.


<p>The two dataframes comprise of same customer_id hence we can join them using the customer_id column. A right join ensures all rows from <code>df2</code> match with rows in <code>df1</code>.</p>

In [17]:
# Perform right join on column named 'id'
df_merged = pd.merge(df_profile, df_transcript, left_on='id', right_on='person', how='right')

# Ensure no rows were dropped during joining
if df_merged.shape[0] == df_transcript.shape[0]:
    print("Right join successful.")

Right join successful.


In [18]:
# Preview dataframe
df_merged.head()

,gender,age,id,became_member_on,income,person,event,value,time
0,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,NaN,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,NaN,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


Next, I'll confirm that the column <code>'id'</code> matches <code>'person'</code>. Then, drop the redundant column.

In [19]:
if df_merged['id'].equals(df_merged['person']):
    df_merged.drop('person', axis=1, inplace=True)
    df_merged.reset_index(drop=True, inplace=True)


<p>The focus now shifts to the column labeled <code>'value'</code>. Through an examination of the dataset on Kaggle, it has been identified that this column typically contains a single key-value pair. However, in some instances, an additional key-value pair is present, which occurs when the column <code>'event'</code> is equal to <code>'offer completed'</code>. This additional key-value pair primarily displays the reward for completing an offer. Since the <code>'reward'</code> column is already present in <code>df0</code>, the second key-value pair is redundant and will be disregarded.</p>
<p>To make the primary key-value pair in <code>'value'</code> more accessible, I will split it into two separate columns. The process involves identifying and printing the unique keys, followed by performing the necessary transformations to separate the key-value pair.</p>
<p>The steps to achieve this are as follows:</p>
<ol>
    <li>Convert the column <code>'value'</code> into a list</li>
    <li>Initialize an empty list to store the processed elements</li>
    <li>While iterating over the elements in the list:</li>
        <ol>
            <li>Convert each string element into a dictionary</li>
            <li>Append the resulting dictionary to the new list</li>
        </ol>
    <li>Initialize an empty set to store unique keys</li>
    <li>Extract the keys from each dictionary in the list and add them to the set (the set will retain only distinct keys)</li>
    <li>Display the distinct keys identified during this process</li>
    <li>Create separate lists for keys and values using list comprehensions</li>
    <li>Construct a new dataframe using these lists</li>
    <li>Concatenate the new dataframe with <code>df3</code></li>
    <li>Remove the original <code>'value'</code> column, as it is no longer needed</li>
</ol>


In [20]:
# Copy df3 and convert the 'value' column to a list
df_merged_c = df_merged.copy()
df_value_list = df_merged_c['value'].tolist()

# Initialize an empty list
value_list = []

# Create list of all dictionaries in 'value'
for element in df_value_list:
    value_dict = eval(element)
    value_list.append(value_dict)

# Initialize an empty set
unique_keys = set()

# Extract unique keys from the key-value pairs
for key in value_list:
    unique_keys.add(next(iter(key)))

# Print unique keys
print(f"The unique keys in column 'value' are: {unique_keys}")

# Create list for keys
value_keys = [next(iter(d)) for d in value_list]

# Create list for values
value_values = [d[key] for d, key in zip(value_list, value_keys)]

# Create new dataframe with separate columns for keys and values
df_keys_values = pd.DataFrame({'offer_id_or_amount': value_keys, 'values': value_values})

# Preview dataframe
print(f"\n{df_keys_values.shape}")
df_keys_values

The unique keys in column 'value' are: {'amount', 'offer id', 'offer_id'}

(306534, 2)


,offer_id_or_amount,values
0,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer id,2906b810c7d4411798c6938adc9daaa5
3,offer id,fafdcd668e3743c1bb461111dcafc2a4
4,offer id,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...
306529,amount,1.59
306530,amount,9.53
306531,amount,3.61
306532,amount,3.53


In [21]:
# Preview dataframe where 'offer_id_or_amount' equals only 'amount'
df_amount = df_keys_values[df_keys_values['offer_id_or_amount'] == 'amount']
print(df_amount.shape)
df_amount.sample(3)

(138953, 2)


,offer_id_or_amount,values
145511,amount,2.73
106118,amount,27.39
291385,amount,10.1


In [22]:
# Concatenate df_keys_values to df3 in a new dataframe
df4 = pd.concat([df_merged, df_keys_values], axis=1)
df4.head()

,gender,age,id,became_member_on,income,event,value,time,offer_id_or_amount,values
0,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,offer id,2906b810c7d4411798c6938adc9daaa5
3,NaN,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,offer id,fafdcd668e3743c1bb461111dcafc2a4
4,NaN,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,offer id,4d5c57ea9a6940dd891ad53e9dbe8da0


In [23]:
# Fixing the inconsistency 'offer id' and 'offer_id'
df4['offer_id_or_amount'] = df4['offer_id_or_amount'].str.replace(' ', '_')

# Confirm fix by printing unique values in the column
print(df4['offer_id_or_amount'].unique())

# Dropping the now redundant 'value' column
df4.drop(columns='value', inplace=True)
df4.tail()

['offer_id' 'amount']


,gender,age,id,became_member_on,income,event,time,offer_id_or_amount,values
306529,M,66,b3a1272bc9904337b331bf348c3e8c17,20180101,47000.0,transaction,714,amount,1.59
306530,M,52,68213b08d99a4ae1b0dcb72aebd9aa35,20180408,62000.0,transaction,714,amount,9.53
306531,F,63,a00058cf10334a308c68e7631c529907,20130922,52000.0,transaction,714,amount,3.61
306532,M,57,76ddbd6576844afe811f1a3c0fbb5bec,20160709,40000.0,transaction,714,amount,3.53
306533,NaN,118,c02b10e8752c4d8e9b73f918558531f7,20151211,NaN,transaction,714,amount,4.05


In [24]:
trans_vs_amt = (df4['event'] == 'transaction') == (df4['offer_id_or_amount'] == 'amount')
print(f"It is {trans_vs_amt.all()} that the numerical entries in 'values' are transaction amounts.")

It is True that the numerical entries in 'values' are transaction amounts.


In [25]:
# Rename column 'id' to 'customer_id'
df4.rename(columns={'id': 'customer_id'}, inplace=True)

# Creating a new dataframe sorted by customer ID number
df5 = df4.sort_values(by=['customer_id'])

<p>There are a number of people in the dataset without demographic information. They are recorded with the <code>'gender'</code> and <code>'income'</code> columns as <code>NaN</code> and the <code>'age'</code> column as <code>118</code>. I will be dropping these individuals from the dataset provided that there is still sufficient data remaining for a robust analysis.</p>

In [26]:
# Display total NaN by column
df5.isna().sum()

,0
gender,33772
age,0
customer_id,0
became_member_on,0
income,33772
event,0
time,0
offer_id_or_amount,0
values,0


Because <code>'age'</code> is equal to <code>118</code> where the <code>'income'</code> and <code>'gender'</code> columns are <code>NaN</code>, I can simply drop the <code>NaN</code> rows, and the 118-year-olds will be dropped as well. I confirm this in the second cell below.

In [27]:
# Drop NA and display new df shape
df5.dropna(axis=0, how='any', inplace=True)
df5.shape

(272762, 9)

In [28]:
# Confirm all 118-year-olds are dropped
(df5['age'] == 118).value_counts()

,count
age,
False,272762



<p>Before proceeding with the final join to create the full dataset, additional modifications will be made to <code>df0</code>. The next step involves parsing the <code>'channels'</code> column, which contains multiple potential values. To handle this, I will apply one-hot encoding.</p>
<p>This process will involve defining a function that creates four new columns, corresponding to the four possible values within the <code>'channels'</code> column. These columns will be initialized with zeroes, and a value of 1 will be assigned if the respective value exists in the original <code>'channels'</code> column.</p>


In [29]:
def encode_channels(df):
    '''
    Performs one-hot encoding on the 'channels' column of the input Pandas DataFrame.

    Parameters:
    The Pandas DataFrame containing the 'channels' column.

    Returns:
    A new DataFrame with the 'channels' column one-hot encoded as four
    separate columns.
    '''

    # Create four new columns of zeroes
    df['channel_web'] = 0
    df['channel_email'] = 0
    df['channel_mobile'] = 0
    df['channel_social'] = 0

    # Iterate over each element in each row, filling corresponding columns with 1s
    for index, row in df.iterrows():
        for element in (row['channels']):
            if element == 'web':
                df.loc[index, 'channel_web'] = 1
            elif element == 'email':
                df.loc[index, 'channel_email'] = 1
            elif element == 'mobile':
                df.loc[index, 'channel_mobile'] = 1
            elif element == 'social':
                df.loc[index, 'channel_social'] = 1

    return df

In [30]:
# Change the elements from one string to a list of strings
df_portfolio['channels'] = df_portfolio['channels'].apply(eval)

# Perform one-hot encoding on df0
df7 = encode_channels(df_portfolio)

# Preview new dataframe
df7.head()

,reward,channels,difficulty,duration,offer_type,id,channel_web,channel_email,channel_mobile,channel_social
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0


Dropping the unnecessary column.

In [31]:
# Drop column 'channels'
df7.drop(columns='channels', inplace=True)

In [32]:
# Perform left join on df5 and df7
df_full = pd.merge(df5, df7, left_on='values', right_on='id', how='left')

# Preview new dataframe
print(df_full.shape)
df_full.head()

(272762, 18)


,gender,age,customer_id,became_member_on,income,event,time,offer_id_or_amount,values,reward,difficulty,duration,offer_type,id,channel_web,channel_email,channel_mobile,channel_social
0,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,transaction,696,amount,18.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,transaction,414,amount,8.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,transaction,690,amount,28.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,offer received,168,offer_id,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,5a8bc65990b245e5a138643cd4eb9837,0.0,1.0,1.0,1.0
4,M,33,0009655768c64bdeb2e877511632db8f,20170421,72000.0,offer received,408,offer_id,f19421c1d4aa40978ebb69ca19b0e20d,5.0,5.0,5.0,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1.0,1.0,1.0,1.0


In [33]:
# Drop the redundant column
df_full.drop(columns='id', inplace=True)

In [34]:
# Rename the column 'became_member_on' to 'member_since'
df_full.rename(columns={'became_member_on': 'member_since'}, inplace=True)

# Rename the column 'time' to 'hours_passed'
df_full.rename(columns={'time': 'hours_passed'}, inplace=True)

# Display all column names
df_full.columns

Index(['gender', 'age', 'customer_id', 'member_since', 'income', 'event',
       'hours_passed', 'offer_id_or_amount', 'values', 'reward', 'difficulty',
       'duration', 'offer_type', 'channel_web', 'channel_email',
       'channel_mobile', 'channel_social'],
      dtype='object')

In [35]:
# Display all distinct values in column 'event'
df_full['event'].unique()

array(['transaction', 'offer received', 'offer viewed', 'offer completed'],
      dtype=object)

In [36]:
# Change spaces to underscores in column 'event'
df_full['event'] = df_full['event'].str.replace(' ', '_')

# Display all distinct values in column 'event'
df_full['event'].unique()

array(['transaction', 'offer_received', 'offer_viewed', 'offer_completed'],
      dtype=object)

In [37]:
# Show data type of the 'member_since' column
df_full['member_since'].dtypes

dtype('int64')

In [38]:
# Print max value in 'member_since'
df_full['member_since'].max()

20180726

In [39]:
# Print min value in 'member_since
df_full['member_since'].min()

20130729

In [40]:
# Convert column to strings
df_full['member_since'] = df_full['member_since'].astype(str)

# # Add dashes
# df_full['member_since'] = df_full['member_since'].str[:4] + '-' \
#                         + df_full['member_since'].str[4:6] + '-' \
#                         + df_full['member_since'].str[6:]

# Convert to datetime
df_full['member_since'] = pd.to_datetime(df_full['member_since'])

# Preview dataframe
df_full.head()

,gender,age,customer_id,member_since,income,event,hours_passed,offer_id_or_amount,values,reward,difficulty,duration,offer_type,channel_web,channel_email,channel_mobile,channel_social
0,M,33,0009655768c64bdeb2e877511632db8f,2017-04-21,72000.0,transaction,696,amount,18.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M,33,0009655768c64bdeb2e877511632db8f,2017-04-21,72000.0,transaction,414,amount,8.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M,33,0009655768c64bdeb2e877511632db8f,2017-04-21,72000.0,transaction,690,amount,28.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M,33,0009655768c64bdeb2e877511632db8f,2017-04-21,72000.0,offer_received,168,offer_id,5a8bc65990b245e5a138643cd4eb9837,0.0,0.0,3.0,informational,0.0,1.0,1.0,1.0
4,M,33,0009655768c64bdeb2e877511632db8f,2017-04-21,72000.0,offer_received,408,offer_id,f19421c1d4aa40978ebb69ca19b0e20d,5.0,5.0,5.0,bogo,1.0,1.0,1.0,1.0


<p>I will now verify the yyyy-mm-dd format, ensuring there are no invalid months or days, by listing unique values for both months and days.</p>

In [41]:
# Display unique months in ascending order
unique_months = df_full['member_since'].dt.month.unique()
print(f'Unique months: {sorted(unique_months)}')

# Display unique days in ascending order
unique_days = df_full['member_since'].dt.day.unique()
print(f'Unique days: {sorted(unique_days)}')

Unique months: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Unique days: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [42]:
# Preview a sample of df_full
print(df_full.shape)
df_full.sample(10)
df_full.to_excel('df_full.xlsx',index=False)

(272762, 17)
